# Let's try training now a model to train on the CIFAR'10 dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)

In [ ]:
import os
from datetime import datetime

import wandb
import torch
from pathlib import Path
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar

from inria.helloworld.trainer_args import TrainerArgs
from inria.helloworld.datamodules import Cifar10DataModule
from inria.helloworld.models import HelloWorldResnet

Let's first check if we have a GPU.

In [ ]:
print(f"GPU available: {torch.cuda.is_available()}")

In [ ]:
DATA_DIR = Path.cwd().parent / "data"
MODELS_DIR = Path.cwd().parent / "models"

In [ ]:
MODEL_CHECKPOINT_DIR = MODELS_DIR / "checkpoints"
BEST_MODEL_DIR = MODELS_DIR / "best_model"

In [ ]:
WANDB_PROJECT = "inria-helloworld-cifar"

In [ ]:
wandb.login()

If we are resuming training we want to check what runs are available in WandB, so we can resume it.

In [ ]:
try:
    for run in wandb.Api().runs(path=os.environ["WANDB_ENTITY"] + "/" + WANDB_PROJECT):
        when = (
            datetime.fromtimestamp(run.summary["_timestamp"]).strftime("%m/%d/%Y, %H:%M:%S")
            if "_timestamp" in run.summary
            else "--"
        )
        print(f"Run id: {run.id} '{run.name}' {when} ({run.state}): {run.url}")
except ValueError as e:
    print(str(e))

In [ ]:
RESUME_RUN_ID = None

In [ ]:
# RESUME_RUN_ID = '2em89whs'  # write here the run that you want to continue

In [ ]:
wandb.init(dir=MODELS_DIR, project=WANDB_PROJECT, resume="allow", id=RESUME_RUN_ID)

In [ ]:
if wandb.run.resumed:
    print("Resumming training from.")
    model = torch.load(wandb.restore("model.ckpt").name)  # setup model
else:
    model = HelloWorldResnet()

In [ ]:
best_models_checkpoint_callback = ModelCheckpoint(
    dirpath=BEST_MODEL_DIR, save_top_k=1, verbose=False, monitor="valid/loss_epoch", mode="min"
)
resume_checkpoint_callback = ModelCheckpoint(dirpath=MODEL_CHECKPOINT_DIR, save_last=True, save_on_train_epoch_end=True)

In [ ]:
early_stop_callback = EarlyStopping(monitor="valid/loss_epoch", min_delta=0.0000001, patience=30, verbose=False, mode="min")

In [ ]:
wandb_logger = WandbLogger(save_dir=MODELS_DIR)

In [ ]:
args = TrainerArgs(
    max_epochs=30,
    log_every_n_steps=10,
    logger=wandb_logger,
    callbacks=[
        best_models_checkpoint_callback,
        resume_checkpoint_callback,
        early_stop_callback,
        TQDMProgressBar(refresh_rate=10),
    ],
)

In [ ]:
args

In [ ]:
cifar = Cifar10DataModule(DATA_DIR)

In [ ]:
args.max_epochs = 30

In [ ]:
trainer = pl.Trainer(**args.to_dict())

In [ ]:
trainer.fit(model, cifar)

In [ ]:
trainer.test(datamodule=cifar, ckpt_path="best")

In [ ]:
wandb.finish()